In [1]:
from website import create_app
from website.models import User, Books, Category, BorrowedBooks, BorrowedBooksDetail

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
app = create_app()

In [48]:
def prepare_data():
    data = []
    headers = BorrowedBooks.query.all()
    for header in headers:
        details = BorrowedBooksDetail.query.filter_by(borrow_id = header.borrow_id)
        for detail in details:
            data.append((header.user_id,detail.book_id))
    books = Books.query.all()
#     for book in books:
#         data.append((0,book.book_id))
    return data

def get_recommendations(user_id, df, num_recommendations):
    # Get the user's book IDs
    user_book_ids = df[df['user_id'] == user_id]['book_id'].tolist()
    
    # Get the books the user has not rated
    unrated_books = df[~df['book_id'].isin(user_book_ids)]
    
    # Create a pivot table of the user's book IDs
    pivot = pd.crosstab(index=df['book_id'], columns=df['user_id'], values=df['book_id'], aggfunc='count', normalize='index')
    
    # Get the cosine similarity between the pivot table and itself
    similarity = cosine_similarity(pivot, pivot)
    
    # Map the similarity to the unrated books
    similarity_map = pd.DataFrame(similarity, index=pivot.index, columns=pivot.index)
    similarity_map = similarity_map.sort_values(by=similarity_map.columns[0], ascending=False)

    # Get the top N most similar books to the pivot
    top_n = similarity_map[unrated_books['book_id']].head(num_recommendations)
    # Get the top N most similar books' IDs
    top_n_book_ids = top_n.index.tolist()
    
#     # Connect to the database
#     engine = create_engine('sqlite:///books.db')
    
#     # Query the database for book information
#     books = pd.read_sql_table('books', engine)
    
#     # Get the top N most similar books' titles
#     top_n_titles = books[books['book_id'].isin(top_n_book_ids)]['title']
    
    return top_n_book_ids

In [49]:
with app.app_context():
    data = prepare_data()
    df = pd.DataFrame(data, columns=['user_id', 'book_id'])
    top_n_book_ids = get_recommendations(1, df, 10)
    datas = books_data(top_n_book_ids)

In [50]:
def books_data(top_n_book_ids):
    data = []
    for ids in top_n_book_ids:
        book_data = Books.query.filter_by(book_id=ids).first()
        if book_data:
            data.append({
                "book_id": book_data.book_id,
                "book_title": book_data.book_title,
                "book_author": book_data.book_author,
                "book_cover_img": book_data.book_cover_img
            })
    return data

In [51]:
datas

[{'book_id': 10,
  'book_title': 'Book 1',
  'book_author': 'dasd',
  'book_cover_img': '/static/uploads/cover_images/HOME_DECOR_INDOOR_WALL_WATER_FALL_IDEAS___WATER_FEATURES_DESIGN_IDEAS___WALL_MOUNTED_IDEAS_water_a.jpg'},
 {'book_id': 15,
  'book_title': 'Book 6',
  'book_author': 'a',
  'book_cover_img': ''},
 {'book_id': 12,
  'book_title': 'Book 3',
  'book_author': '3',
  'book_cover_img': ''},
 {'book_id': 16,
  'book_title': 'Book 8',
  'book_author': 'book 8',
  'book_cover_img': ''}]

In [67]:
user_id = 4
num_recommendations = 10

In [68]:
# Get the user's book IDs
user_book_ids = df[df['user_id'] == user_id]['book_id'].tolist()

In [69]:
user_book_ids

[15, 12]

In [70]:
# Get the books the user has not rated
unrated_books = df[~df['book_id'].isin(user_book_ids)]

In [71]:
unrated_books

,user_id,book_id
0,1,16
4,5,10


In [72]:
# Create a pivot table of the user's book IDs
pivot = pd.crosstab(index=df['book_id'], columns=df['user_id'], values=df['book_id'], aggfunc='count', normalize='index')

In [73]:
pivot

user_id,1,4,5
book_id,,,
10,0.0,0.0,1.0
12,0.0,1.0,0.0
15,0.0,0.5,0.5
16,1.0,0.0,0.0


In [74]:
# Get the cosine similarity between the pivot table and itself
similarity = cosine_similarity(pivot, pivot)

In [75]:
similarity

array([[1.        , 0.        , 0.70710678, 0.        ],
       [0.        , 1.        , 0.70710678, 0.        ],
       [0.70710678, 0.70710678, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [76]:
# Map the similarity to the unrated books
similarity_map = pd.DataFrame(similarity, index=pivot.index, columns=pivot.index)
similarity_map = similarity_map.sort_values(by=similarity_map.columns[0], ascending=False)

In [77]:
similarity_map

book_id,10,12,15,16
book_id,,,,
10,1.000000,0.000000,0.707107,0.0
15,0.707107,0.707107,1.000000,0.0
12,0.000000,1.000000,0.707107,0.0
16,0.000000,0.000000,0.000000,1.0


In [ ]:
# Get the user's book IDs
user_book_ids = df[df['user_id'] == user_id]['book_id'].tolist()

# Get the books the user has not rated
unrated_books = df[~df['book_id'].isin(user_book_ids)]

# Create a pivot table of the user's book IDs
pivot = pd.crosstab(index=df['book_id'], columns=df['user_id'], values=df['book_id'], aggfunc='count', normalize='index')

# Get the cosine similarity between the pivot table and itself
similarity = cosine_similarity(pivot, pivot)

# Map the similarity to the unrated books
similarity_map = pd.DataFrame(similarity, index=pivot.index, columns=pivot.index)
similarity_map = similarity_map.sort_values(by=similarity_map.columns[0], ascending=False)

# Get the top N most similar books to the pivot
top_n = similarity_map[unrated_books['book_id']].head(num_recommendations)
# Get the top N most similar books' IDs
top_n_book_ids = top_n.index.tolist()


In [ ]:
top_n = similarity_map[unrated_books['book_id']].sort_values(15,ascending=False)

In [ ]:
top_n,similarity,similarity_map

In [ ]:
# Get the user's books
user_books = df[df['user_id'] == user_id]

# Get the books the user has not rated
unrated_books = df[~df['book_id'].isin(user_books['book_id'])]

# Create a pivot table of the user's books
pivot = pd.pivot_table(df, index='book_id', columns='user_id', values='book_id')

# Get the cosine similarity between the pivot table and itself
similarity = cosine_similarity(pivot, pivot)

# Map the similarity to the unrated books
similarity_map = pd.DataFrame(similarity, index=pivot.index, columns=pivot.index)

# Get the top N most similar books to the pivot
#top_n = similarity_map[unrated_books['book_id']].sort_values(user_id, ascending=False).head(num_recommendations)


In [ ]:
!pip install scikit-learn

In [ ]:
similarity_map = pd.DataFrame(similarity, index=pivot.index, columns=pivot.columns)


In [ ]:
import pandas as pd
from scipy.spatial.distance import cosine

def prepare_data():
    return [
        [1, 1, 5],
        [1, 2, 4],
        [1, 3, 2],
        [2, 1, 4],
        [2, 2, 3],
        [2, 3, 5],
        [3, 1, 2],
        [3, 2, 1],
        [3, 3, 5],
    ]

def get_recommendations(user_id, df, num_recommendations):
    pivot = pd.pivot_table(df, values='rating', index='user_id', columns='book_id')
    user_ratings = pivot.loc[user_id].dropna()
    unrated_books = pivot.loc[:, pivot.loc[user_id].isna()].dropna(axis=1)
    similarity = 1 - cosine(pivot.loc[user_id].values.reshape(1, -1), pivot.T)
    similarity_map = pd.Series(similarity, index=pivot.index)
    top_n = similarity_map.sort_values(ascending=False).head(num_recommendations+1)
    top_n = top_n.drop(user_id)
    top_n_book_ids = top_n.index.tolist()
    return top_n_book_ids

data = prepare_data()
df = pd.DataFrame(data, columns=['user_id', 'book_id', 'rating'])
top_n_book_ids = get_recommendations(1, df, 2)
print(top_n_book_ids)
